# Project 5

## Notebook 1

### Question 1: Processing this data to recover some song statistics. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import pickle
import tarfile
import io

#### Importing the data files

In [2]:
import pandas as pd
ice_fulltable = pd.read_table('userid-timestamp-artid-artname-traid-traname.tsv', sep='\t',error_bad_lines=False,nrows=200000)
ice_fulltable.columns = ['User', 'Date', 'Standard','Artist','Label', 'SongName']
ice_profile = pd.read_table('userid-profile.tsv', sep='\t',error_bad_lines=False)


In [3]:
ice_fulltable.shape
ice_fulltable.head()

,User,Date,Standard,Artist,Label,SongName
0,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
1,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
2,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
4,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)


In [4]:
ice_profile.columns=['User','Gender','Age','Country','Registered']
ice_profile.head()
ice_profile.shape

(992, 5)

#### Merging the data

In [5]:
Merged=ice_fulltable.merge(ice_profile, left_on='User', right_on='User', how='inner')
Merged

pickle_out = open('data.pk2', 'wb')
pickle.dump(Merged, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()


#### Dropping the missing data

In [6]:
Merged=Merged.dropna()
Merged.shape

(51945, 10)

#### The number of times a song has been played

In [7]:
merged1=Merged['SongName'].value_counts()
merged2=merged1.to_frame(name='dj').reset_index(level=[0])
merged2.columns=['SongName','Song Count']
merged2.head()

,SongName,Song Count
0,Heartless,2029
1,See You In My Nightmares,1979
2,Say You Will,1976
3,Love Lockdown,1970
4,Welcome To Heartbreak (Feat. Kid Cudi),1970


#### Percentage of the plays that came from users who self-identified as male

In [8]:
merged3=Merged.groupby(['SongName','Gender']).size()
merged3 = merged3.reset_index(level=[0,1])
merged3.columns=['SongName','Gender','Gender Played']
merged4=merged3[merged3['Gender']=='m']
merged4.columns=['SongName','Gender','Male Played']
merged4.head()

,SongName,Gender,Male Played
0,#34,m,1
1,& Down (Siriusmo Vs. Boys Noize Mix),m,3
2,'98 Freestyle,m,1
3,(Getting/Giving The Lock),m,2
4,(I Am) Made From The Wires,m,6


In [9]:
PercentData=pd.DataFrame(columns=['SongName','Male Artists'])

for song in set(merged4['SongName']):
    if(sum(merged4['SongName']==song)>0):
        track_user_male_count=merged4[merged4['SongName']==song].iloc[0][2]
        track_user_total_count=merged2[merged2['SongName']==song].iloc[0][1]
        track_user_male_percentage=((track_user_male_count/track_user_total_count)*100)
    else:
        track_user_male_percentage=0
    PercentData.loc[len(PercentData)]=[song,track_user_male_percentage]
PercentData.head()



,SongName,Male Artists
0,That Is Why,100
1,No Time,100
2,Am I Wrong (Extended),100
3,Rise Up,100
4,The Archer'S Bows Have Broken,100


#### Average age of the listener

In [10]:
Group1=Merged.groupby(['SongName','User','Age']).size()
Group1 = Group1.reset_index(level=[0,1,2])
Group1.columns=['SongName','User','Age','Song Count']
AgeGroup=Group1[Group1['Age']!=0]
AgeGroup.head()

,SongName,User,Age,Song Count
0,#34,user_000003,22.0,1
1,& Down (Siriusmo Vs. Boys Noize Mix),user_000003,22.0,3
2,'98 Freestyle,user_000003,22.0,1
3,(Getting/Giving The Lock),user_000003,22.0,2
4,(I Am) Made From The Wires,user_000003,22.0,6


In [11]:
DF1=pd.DataFrame(columns=['SongName','AgeAvg'])
for song in set(merged2['SongName']):
    SongCount1=sum(AgeGroup['SongName']==song)
    if(SongCount1>0):
        AgeAvg1=sum(AgeGroup[AgeGroup['SongName']==song]['Age'].astype(int))
        AgeAvg2=AgeAvg1/SongCount1
    else:
        AgeAvg=0
    DF1.loc[len(DF1)]=[song,AgeAvg2]
DF1.head()

,SongName,AgeAvg
0,That Is Why,22
1,No Time,22
2,Finale: Allegro Vivacissimo,22
3,Rise Up,23
4,Appelez Moi Kenza,23


#### Percentage of plays that came from a user who played the song exactly once

In [12]:
Model1=Group1[Group1['Song Count']==1]
Model1.head()

,SongName,User,Age,Song Count
0,#34,user_000003,22.0,1
2,'98 Freestyle,user_000003,22.0,1
8,"(Shake, Shake, Shake) Shake Your Booty",user_000003,22.0,1
11,...And The Day Turned To Night,user_000003,22.0,1
17,10 Years Today,user_000003,22.0,1


In [13]:
Node1=pd.DataFrame(columns=['SongName','UserSongPercent'])

for song in set(merged2['SongName']):
    UserSongCount=sum(Model1['SongName']==song)
    if(UserSongCount>0):
        TotalsongsCount=merged2[merged2['SongName']==song].iloc[0][1]
        UserSongPercent=(UserSongCount/TotalsongsCount)*100
    else:
        UserSongPercent=0
    Node1.loc[len(Node1)]=[song,UserSongPercent]
Node1.head()

,SongName,UserSongPercent
0,That Is Why,100
1,No Time,0
2,Finale: Allegro Vivacissimo,0
3,Rise Up,0
4,Appelez Moi Kenza,100


#### Percentage of plays that came from a user who played the song at least five times 

In [14]:
Part1=Group1[Group1['Song Count']>5]
Part1.head()

,SongName,User,Age,Song Count
4,(I Am) Made From The Wires,user_000003,22.0,6
12,...But Home Is Nowhere,user_000003,22.0,8
13,...The Frequency Hostage,user_000003,22.0,6
15,1.618,user_000003,22.0,8
23,12D3,user_000003,22.0,8


In [15]:
Qart1=pd.DataFrame(columns=['SongName','UserFiveTrackPercent'])
for song in set(merged2['SongName']):
    UserFiveTrackPercent=sum(Part1[Part1['SongName']==song]['Song Count'].astype(int))
    if(UserFiveTrackPercent>0):
        TotalSongCount=merged2[merged2['SongName']==song].iloc[0][1]
        TotalSongPercent=(UserFiveTrackPercent/TotalSongCount)*100
    else:
        TotalSongPercent=0
    Qart1.loc[len(Qart1)]=[song,UserFiveTrackPercent]
Qart1.head()

,SongName,UserFiveTrackPercent
0,That Is Why,0
1,No Time,0
2,Finale: Allegro Vivacissimo,0
3,Rise Up,0
4,Appelez Moi Kenza,0


#### Generating some descriptive statistics.

In [16]:
TMerge=merged2.merge(PercentData, left_on='SongName', right_on='SongName', how='inner')
TMerge

,SongName,Song Count,Male Artists
0,Heartless,2029,100
1,See You In My Nightmares,1979,100
2,Say You Will,1976,100
3,Love Lockdown,1970,100
4,Welcome To Heartbreak (Feat. Kid Cudi),1970,100
5,Coldest Winter,1961,100
6,Paranoid (Feat. Mr. Hudson),1960,100
7,Amazing (Feat. Young Jeezy),1960,100
8,Pinocchio Story (Freestyle Live From Singapore),1951,100
9,Bad News,1936,100


In [17]:
TMerge=TMerge.merge(DF1, left_on='SongName', right_on='SongName', how='inner')
TMerge

,SongName,Song Count,Male Artists,AgeAvg
0,Heartless,2029,100,23
1,See You In My Nightmares,1979,100,23
2,Say You Will,1976,100,23
3,Love Lockdown,1970,100,23
4,Welcome To Heartbreak (Feat. Kid Cudi),1970,100,23
5,Coldest Winter,1961,100,23
6,Paranoid (Feat. Mr. Hudson),1960,100,23
7,Amazing (Feat. Young Jeezy),1960,100,23
8,Pinocchio Story (Freestyle Live From Singapore),1951,100,23
9,Bad News,1936,100,23


In [18]:
TMerge=TMerge.merge(Node1, left_on='SongName', right_on='SongName', how='inner')
TMerge

,SongName,Song Count,Male Artists,AgeAvg,UserSongPercent
0,Heartless,2029,100,23,0
1,See You In My Nightmares,1979,100,23,0
2,Say You Will,1976,100,23,0
3,Love Lockdown,1970,100,23,0
4,Welcome To Heartbreak (Feat. Kid Cudi),1970,100,23,0
5,Coldest Winter,1961,100,23,0
6,Paranoid (Feat. Mr. Hudson),1960,100,23,0
7,Amazing (Feat. Young Jeezy),1960,100,23,0
8,Pinocchio Story (Freestyle Live From Singapore),1951,100,23,0
9,Bad News,1936,100,23,0


In [19]:
TMerge=TMerge.merge(Qart1, left_on='SongName', right_on='SongName', how='inner')
TMerge

,SongName,Song Count,Male Artists,AgeAvg,UserSongPercent,UserFiveTrackPercent
0,Heartless,2029,100,23,0,2029
1,See You In My Nightmares,1979,100,23,0,1979
2,Say You Will,1976,100,23,0,1976
3,Love Lockdown,1970,100,23,0,1970
4,Welcome To Heartbreak (Feat. Kid Cudi),1970,100,23,0,1970
5,Coldest Winter,1961,100,23,0,1961
6,Paranoid (Feat. Mr. Hudson),1960,100,23,0,1960
7,Amazing (Feat. Young Jeezy),1960,100,23,0,1960
8,Pinocchio Story (Freestyle Live From Singapore),1951,100,23,0,1951
9,Bad News,1936,100,23,0,1936


#### Exporting the generated dataframe using pickle for posterity

In [20]:
pickle_out = open('data.pkl', 'wb')
pickle.dump(TMerge, pickle_out, pickle.HIGHEST_PROTOCOL)
pickle_out.close()


In [21]:
Yart1=TMerge.sort_values(by=['UserFiveTrackPercent'],ascending=True)

In [22]:
pickle_in = open("data.pkl", "rb")
SeaIce = pickle.load(pickle_in)

In [23]:
print (type(SeaIce))

<class 'pandas.core.frame.DataFrame'>
